<a href="https://colab.research.google.com/github/AryanNanda17/GestureSense/blob/Mihir/PyTorchClassApiModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.models as models

#load the dataset here after preprocessing, define num_classes, and segregate data into train_loader and test_loader.

class GestureRecognitionModel(nn.Module):
    def __init__(self, num_classes):
        super(GestureRecognitionModel, self).__init()

        self.conv1 = nn.Conv3d(in_channels=3, out_channels=64, kernel_size=(3, 3, 3), stride=(1, 2, 2))
        self.maxpool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.conv2 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=(3, 3, 3))
        self.maxpool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv3 = nn.Conv3d(in_channels=128, out_channels=128, kernel_size=(3, 3, 3))
        self.conv4 = nn.Conv3d(in_channels=128, out_channels=128, kernel_size=(3, 3, 3))
        self.maxpool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.batchnorm = nn.BatchNorm3d(128)

        self.lstm = nn.LSTM(input_size=128, hidden_size=128, num_layers=2, batch_first=True, dropout=0.5)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.maxpool1(torch.relu(self.conv1(x)))
        x = self.maxpool2(torch.relu(self.conv2(x)))
        x = self.maxpool3(torch.relu(self.conv4(self.conv3(x))))

        x = x.view(x.size(0), x.size(1), -1)  # Flatten to (batch_size, sequence_length, features)

        lstm_out, _ = self.lstm(x)
        x = self.dropout(lstm_out[:, -1, :])  # Select the last LSTM output
        x = self.fc(x)
        x = self.softmax(x)
        return x

# Create an instance of the model
model = GestureRecognitionModel(num_classes=num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}] - Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%')

# Testing loop (optional)
model.eval()  # Set the model to evaluation mode
test_correct = 0
test_total = 0


with torch.no_grad():
    for inputs, labels in test_loader:  # You should define a DataLoader for testing data
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')


AttributeError: ignored